<h1> LABORATORIO 3</h1>
<h2>ALUMNO: LOAIZA CORONADO RODRIGO JAIRO </h2>
<h2>CARRERA ING. DE SISTEMAS</h2>
<h3>FECHA: 19/03/2024</h3>

<h3> Clasificacion Multiclase </h3>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# utilizado para la manipulación de directorios y rutas
import os

#importamos pandas para el manejo del dataset, y separarlos dentro de una matriz
import pandas as pd
# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:

input_layer_size  = 18
# tengo 6 clases
num_labels = 6
#se pone 7 a pesar de haber 6 clases con esto el 0 no se toma en cuenta (como es uno contra todos hay que realizar este artificio para considerar todas las clases u otra opcion es convertir el 6 en 0)(el argmmax nos da 0 1 2 y para tomar en cuenta las 3 clases aumentamos 1 mas y quitamos el 0)
data = np.loadtxt('/content/drive/MyDrive/SIS420 Inteligencia Artificial/machine learning/datasets/Datos.csv', delimiter=',', skiprows=1, usecols=range(1, input_layer_size+1))


# Separar las características (X) de las etiquetas (y)
X = data[:, :-1]  # Todas las filas, todas las columnas excepto la última
y = data[:, -1]   # Todas las filas, última columna

# Número de ejemplos
m = y.size

In [ ]:
print(X[0, :])
print('---------------------- ')  # Imprime la primera fila de características
print(y)             # Imprime las etiquetas


[ 0.          1.          0.          2.          0.          0.
  3.          3.          0.14893617  0.32075472  0.18656716  0.5
  0.66666667  0.5         0.          0.5        24.38652644]
---------------------- 
[1. 1. 1. ... 5. 3. 5.]


In [ ]:
# Calcular el conteo de cada clase
clases, conteo = np.unique(y, return_counts=True)

# Imprimir el conteo de cada clase
for clase, count in zip(clases, conteo):
    print(f"Clase {int(clase)}: {count} ejemplos")

Clase 0: 2795 ejemplos
Clase 1: 3369 ejemplos
Clase 2: 2717 ejemplos
Clase 3: 2812 ejemplos
Clase 4: 3261 ejemplos
Clase 5: 3545 ejemplos
Clase 6: 4370 ejemplos


In [ ]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [ ]:
X_norm, mu, sigma = featureNormalize(X)

In [ ]:
m, n = X.shape
X = X_norm

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

<h2>COSTO POR EL GRADIENTE</h2>

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    m = y.size
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

<h2>Clasificacion One-vs-all</h2>

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta



In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)
    return p
    # Calcula el número de datos para el conjunto de entrenamiento (80%)
num_train = int(0.8 * 22800)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test = X[:num_train], X[num_train:]
y_train, y_test = y[:num_train], y[num_train:]
# Entrenar el modelo utilizando todos los datos
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
all_theta_train = oneVsAll(X_train, y_train, num_labels, lambda_)

In [ ]:
print(all_theta)

[[-1.59091712e+01  5.78705507e-02 -5.43960062e-02  1.01379673e-02
  -1.30830373e-01 -1.82058826e-01 -1.87783887e-02  1.38204178e-01
  -4.06642749e-01 -3.55117032e-01  6.35002321e-01 -5.37246909e+00
   2.24791332e-01  1.41399021e-01  1.54725268e-01 -9.18285716e-03
   2.83286726e-01 -6.54852553e+00]
 [-2.72644307e+00  4.06586888e-01  5.36738441e-02 -4.14434887e-02
  -1.15691985e-01  1.14343086e-01 -1.05330497e-01  3.80749015e-02
   1.73220635e-01 -1.81397179e-01 -3.19265523e-01 -1.13491301e+00
  -2.67585886e-02  2.35157938e-01 -1.15253494e-01  1.02933358e-01
  -2.05830220e-01 -4.86631239e-01]
 [-2.45961261e+00  2.32951741e-01 -1.59542021e-02  1.27021597e-01
   3.74929868e-01 -1.21926565e-02  2.49499485e-01 -4.93098531e-01
  -4.34860599e-02 -1.14130704e-02  6.84731877e-01 -3.29286580e+00
  -2.68279672e-01 -2.78133776e-01  2.53069008e-01  1.23394069e-01
  -1.73215777e-01  2.19167231e+00]
 [-2.50858411e+00  5.19425225e-01  5.99005518e-01 -4.63179224e-01
   1.77908191e-01 -5.54471917e-02 -2.

<h2>Prediccion One-vs-all </h2>

In [ ]:
# Predecir las clases utilizando el conjunto de entrenamiento
pred_train = predictOneVsAll(all_theta_train, X_train)

# Calcular la precisión del conjunto de entrenamiento
accuracy_train = np.mean(pred_train == y_train) * 100
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(accuracy_train))
# Predecir las clases utilizando el conjunto de prueba
pred_test = predictOneVsAll(all_theta_train, X_test)

# Calcular la precisión del conjunto de prueba
accuracy_test = np.mean(pred_test == y_test) * 100
print('Precisión del conjunto de prueba: {:.2f}%'.format(accuracy_test))

# Aquí puedes realizar cualquier otra evaluación o visualización necesaria

Precisión del conjunto de entrenamiento: 57.02%
Precisión del conjunto de prueba: 57.46%


In [ ]:
# Ejemplo de predicción para nuevos datos
X_prueba = np.array([[ 1, 1, 0, 2, 0, 0, 3, 1, 0.14893617021276595, 0.3207547169811322, 0.18656716417910446, 0.5, 0.6666666666666667, 0.5, 0.0, 0.5, 10.38652644413961],
                     [ 1, 1, 0, 2, 1, 1, 2, 3, 0.14893617021276595, 0.1320754716981134, 0.12686567164179102, 1.0, 0.6666666666666667, 1.0, 1.0, 0.0, 24.238227146814403],
                     [ 2, 1, 0, 2, 0, 0, 1, 3, 0.19148936170212766, 0.6603773584905661, 0.2835820895522388, 0.5, 0.6666666666666667, 0.5, 0.6666666666666666, 0.5, 23.76543209876543],
                     [ 0, 0, 0, 0, 1, 1, 1, 0, 0.276595744680851, 0.6603773584905661, 0.3582089552238806, 1.0, 0.6666666666666667, 0.5, 0.6666666666666666, 0.0, 12.85185185185185],
                     [1,1,1,2,0,0,2,3,0.1761246595744681,0.6437245283018869,0.492814947761194,0.9989755,0.6666666666666667,0.517075,0.6666666666666666,0.2543315,32.7391647533067]])
p_prueba = predictOneVsAll(all_theta, X_prueba)
print("Predicciones para nuevos datos:", p_prueba)


Predicciones para nuevos datos: [5 2 2 4 2]


In [ ]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))

(22869, 17)
Precision del conjuto de entrenamiento: 57.01%


In [ ]:
# Código Agregado
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

# displayData(X[1002:1003, :])
print(y[17:150])

(140, 17)
(140, 18)
[3 2 1 2 3 1 5 4 1 3 3 3 1 4 3 1 2 0 2 2 4 3 2 3 1 3 1 1 1 2 2 1 4 1 3 3 4
 4 1 1 1 0 0 2 1 1 3 3 1 0 1 1 2 0 1 4 4 4 2 3 4 0 2 3 2 0 0 4 4 3 1 3 4 0
 3 3 4 4 2 2 1 1 3 1 0 3 1 0 0 2 2 3 2 1 4 2 2 3 3 4 1 4 1 1 2 0 1 4 1 1 4
 4 0 3 1 1 1 2 1 2 1 1 2 3 5 2 3 3 1 1 0 1 1 2 2 2 1 3 1 2]
[4. 3. 2. 3. 4. 1. 4. 1. 1. 1. 1. 1. 1. 2. 3. 1. 3. 1. 3. 1. 1. 1. 3. 2.
 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 2. 3. 4. 5. 1. 3. 0. 1. 1. 3. 0. 0. 3. 4. 1. 1. 3. 4. 0. 3. 1. 1. 2. 1.
 3. 5. 1. 2. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 4. 1. 1. 2. 4. 4. 1. 1. 1.
 1. 1. 1. 4. 3. 1. 3. 2. 3. 0. 1. 3. 2. 1. 1. 1. 3. 1. 1. 1. 1. 4. 2. 1.
 4. 1. 1. 0. 1. 4. 2. 2. 2. 1. 4. 1. 4.]
